In [4]:
import pandas as pd
import time
import numpy as np

In [5]:
counts_per_hourdf = pd.read_csv('Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv')
Sensor_Locationsdf = pd.read_csv ('Pedestrian_Counting_System_-_Sensor_Locations.csv')

In [6]:
counts_per_hourdf.shape,Sensor_Locationsdf.shape

((3048367, 10), (64, 11))

In [7]:
counts_per_hourdf.head()

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts
0,2887628,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,34,Flinders St-Spark La,300
1,2887629,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,39,Alfred Place,604
2,2887630,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,37,Lygon St (East),216
3,2887631,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,40,Lonsdale St-Spring St (West),627
4,2887632,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,36,Queen St (West),774


In [10]:
SensorSubsetDF = Sensor_Locationsdf[['sensor_id','sensor_description']]
Sensor_Locationsdf.head()

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,59,Building 80 RMIT,RMIT_T,2019/02/13,A,NaN,North,South,-37.808256,144.963049,"(-37.80825648, 144.96304859)"
1,23,Spencer St-Collins St (South),Col623_T,2013/09/02,A,NaN,East,West,-37.819093,144.954527,"(-37.81909256, 144.95452749)"
2,20,Chinatown-Lt Bourke St (South),LtB170_T,2013/09/06,A,NaN,East,West,-37.811729,144.968247,"(-37.81172913, 144.9682466)"
3,34,Flinders St-Spark La,Fli32_T,2014/06/08,A,NaN,East,West,-37.815380,144.974150,"(-37.81537985, 144.9741505)"
4,57,Bourke St Bridge,BouBri_T,2018/08/13,A,NaN,West,East,-37.817673,144.950256,"(-37.8176735, 144.95025595)"


In [25]:
counts_per_hourdf['Date_Time'] =  pd.to_datetime(counts_per_hourdf['Date_Time'], infer_datetime_format=True)

Timestamp('2019-11-01 17:00:00')

In [28]:
counts_per_hourdf['DateKey'] = counts_per_hourdf.Date_Time.apply(lambda x: x.strftime('%Y%m%d')).astype(int)

ID                        int64
Date_Time        datetime64[ns]
Year                      int64
Month                    object
Mdate                     int64
Day                      object
Time                      int64
Sensor_ID                 int64
Sensor_Name              object
Hourly_Counts             int64
dtype: object

In [39]:
#counts_per_hourdf.head()
CountSubsetDF = counts_per_hourdf[['Sensor_ID','Hourly_Counts','Year','Month','DateKey']]
CountSubsetDF.head()

,Sensor_ID,Hourly_Counts,Year,Month,DateKey
0,34,300,2019,November,20191101
1,39,604,2019,November,20191101
2,37,216,2019,November,20191101
3,40,627,2019,November,20191101
4,36,774,2019,November,20191101


In [45]:
#SensorSubsetDF.head()
SelectedDF = pd.merge(SensorSubsetDF, CountSubsetDF, left_on='sensor_id', right_on='Sensor_ID')
del SelectedDF['Sensor_ID']

,sensor_id,sensor_description,Hourly_Counts,Year,Month,DateKey
3048362,39,Alfred Place,162,2020,February,20200229
3048363,39,Alfred Place,151,2020,February,20200229
3048364,39,Alfred Place,141,2020,February,20200229
3048365,39,Alfred Place,84,2020,February,20200229
3048366,39,Alfred Place,72,2020,February,20200229


## Extract StatsNeed to show contrived of expected outputs.

- Top 10 (most pedestrians) locations by day

- Top 10 (most pedestrians) locations by month

 

## Load- Data into S3 in an appropriate format for future querying

Statistical data into an appropriate data store

# Top 10 (most pedestrians) locations by day

In [78]:
#Top 10 (most pedestrians) locations by day
# Group by Hourlycount on Datekey and location
Q1DF = SelectedDF.groupby(['DateKey', 'sensor_id','sensor_description']).agg({'Hourly_Counts': "sum"})
Q1DF.reset_index(inplace = True) 

In [79]:
Q1DF["rank"] = Q1DF.groupby('DateKey')['Hourly_Counts'].rank(ascending=False)
Q1DF = Q1DF.sort_values(by=['DateKey','rank'], ascending=True)
Q1DF = Q1DF[Q1DF['rank'] <=10]
del Q1DF['rank']
Q1DF.head()

,DateKey,sensor_id,sensor_description,Hourly_Counts
2,20090501,4,Town Hall (West),45185
0,20090501,1,Bourke Street Mall (North),36869
4,20090501,6,Flinders Street Station Underpass,29015
1,20090501,2,Bourke Street Mall (South),27587
3,20090501,5,Princes Bridge,25590


In [80]:
Q1DF.shape

(39569, 4)

## Top 10 (most pedestrians) locations by month

In [81]:
Q2DF = SelectedDF.groupby(['Year','Month', 'sensor_id','sensor_description']).agg({'Hourly_Counts': "sum"})
Q2DF.reset_index(inplace = True) 
Q2DF['rank'] = Q2DF.groupby(['Year','Month'])['Hourly_Counts'].rank(ascending=False)
Q2DF = Q2DF.sort_values(by = ['Year','Month','rank'], ascending = True)
Q2DF = Q2DF[Q2DF  ['rank'] <= 10]
del Q2DF['rank']
Q2DF.head()

,Year,Month,sensor_id,sensor_description,Hourly_Counts
3,2009,August,4,Town Hall (West),1050461
1,2009,August,2,Bourke Street Mall (South),847853
2,2009,August,3,Melbourne Central,827432
5,2009,August,6,Flinders Street Station Underpass,721539
4,2009,August,5,Princes Bridge,700272


In [82]:
Q2DF

,Year,Month,sensor_id,sensor_description,Hourly_Counts
3,2009,August,4,Town Hall (West),1050461
1,2009,August,2,Bourke Street Mall (South),847853
2,2009,August,3,Melbourne Central,827432
5,2009,August,6,Flinders Street Station Underpass,721539
4,2009,August,5,Princes Bridge,700272
0,2009,August,1,Bourke Street Mall (North),560625
11,2009,August,13,Flagstaff Station,540355
13,2009,August,15,State Library,539908
14,2009,August,16,Australia on Collins,489810
15,2009,August,17,Collins Place (South),326793
